In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import csv
import cv2
import numpy as np
import random
import os
import time
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms, utils, io
# !pip install GPUtil
# from GPUtil import showUtilization as gpu_usage
# gpu_usage()         

In [4]:
TRAIN_PATH = "/content/drive/MyDrive/captcha-hacker/train"
TEST_PATH = "/content/drive/MyDrive/captcha-hacker/test"
# device = "cpu"
device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster

In [5]:
class Task1Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task1")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
#         img = cv2.imread(f"{self.root}/{filename}")
        img = io.read_image(f"{self.root}/{filename}")
#         img = cv2.resize(img, (32, 3))
#         img = np.mean(img, axis=2)
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
            return torch.FloatTensor(img / 256), int(label)

    def __len__(self):
        return len(self.data)

In [6]:
# class Model(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(1024, 512),
#             nn.LeakyReLU(),
#             nn.Linear(512, 10)
#         )
        
        
#     def forward(self, x):
#         b, h, w = x.shape
#         x = x.view(b, h*w)
#         return self.layers(x)

class ResNet18_1(nn.Module):
    def __init__(self):
        super(ResNet18_1, self).__init__()

        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 10)

    def forward(self, x):
        return self.model(x)

In [7]:

torch.cuda.empty_cache()
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
alphabet = 'abcdefghijklmnopqrstuvwxyz0123456789'
alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','0','1','2','3','4','5','6','7','8','9']
alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
dic={'alphabet':alphabets}
labelencoder = LabelEncoder()
data_le=pd.DataFrame(dic)
data_le['alphabet'] = labelencoder.fit_transform(data_le['alphabet'])
data_le
onehotencoder = OneHotEncoder()
data_str_ohe=onehotencoder.fit_transform(data_le).toarray()
pd.DataFrame(data_str_ohe)
# data_dum = pd.get_dummies(data_le)
# pd.DataFrame(data_dum)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
class Task2Dataset(Dataset):
    def __init__(self, data, root, transform=None, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task2")]
        print(len(self.data))
#         self.new_data = make_dataset(self.data, alphabet)
#         print(len(self.new_data))
        self.return_filename = return_filename
        self.root = root
        self.transform = transform
#       self.samples = make_dataset(self.data_path,self.ans_path,self.alphabet)
    
    def __getitem__(self, index):
        filename, label = self.data[index]
#         img = cv2.imread(f"{self.root}/{filename}")
        img = io.read_image(f"{self.root}/{filename}")
#         img = cv2.resize(img, (32, 3))
#         img = np.mean(img, axis=2)s
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
#             label = torch.Tensor(label)
            t=[]
            for char in str(label):
                v = [0] * 36
                v[alphabet.find(char)] = 1
                t += v
            return torch.FloatTensor(img / 256), torch.Tensor(t)
    def __len__(self):
        return len(self.data)

In [9]:
class ResNet18_2(nn.Module):
    def __init__(self):
        super(ResNet18_2, self).__init__()

        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 72)

    def forward(self, x):
#         b, h, w = x.shape
#         x = x.view(b, h*w)
# #         return self.model(x)
        return self.model(x)

In [10]:
torch.cuda.empty_cache()
alphabet = 'abcdefghijklmnopqrstuvwxyz0123456789'
alphabets = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','0','1','2','3','4','5','6','7','8','9']
alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
dic={'alphabet':alphabets}
labelencoder = LabelEncoder()
data_le=pd.DataFrame(dic)
data_le['alphabet'] = labelencoder.fit_transform(data_le['alphabet'])
data_le
onehotencoder = OneHotEncoder()
data_str_ohe=onehotencoder.fit_transform(data_le).toarray()
pd.DataFrame(data_str_ohe)
# data_dum = pd.get_dummies(data_le)
# pd.DataFrame(data_dum)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
class Task3Dataset(Dataset):
    def __init__(self, data, root, transform=None, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task3")]
        print(len(self.data))
#         self.new_data = make_dataset(self.data, alphabet)
#         print(len(self.new_data))
        self.return_filename = return_filename
        self.root = root
        self.transform = transform
#       self.samples = make_dataset(self.data_path,self.ans_path,self.alphabet)
    
    def __getitem__(self, index):
        filename, label = self.data[index]
#         img = cv2.imread(f"{self.root}/{filename}")
        img = io.read_image(f"{self.root}/{filename}")
#         img = cv2.resize(img, (32, 3))
#         img = np.mean(img, axis=2)s
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
#             label = torch.Tensor(label)
            t=[]
            for char in str(label):
                v = [0] * 36
                v[alphabet.find(char)] = 1
                t += v
            return torch.FloatTensor(img / 256), torch.Tensor(t)
    def __len__(self):
        return len(self.data)

In [12]:
class ResNet18_3(nn.Module):
    def __init__(self):
        super(ResNet18_3, self).__init__()

        self.model = models.resnet18(pretrained=True)

        self.model.fc = nn.Linear(512, 144)

    def forward(self, x):
        return self.model(x)

In [ ]:
torch.cuda.empty_cache()
test_data = []
alphabet = 'abcdefghijklmnopqrstuvwxyz0123456789'
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
image_change = transforms.Compose([transforms.Resize((384, 288)), transforms.Normalize(mean, std)])

with open(f'/content/drive/MyDrive/captcha-hacker/sample_submission.csv', newline='') as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)

test_ds_1 = Task1Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl_1 = DataLoader(test_ds_1, batch_size=32, num_workers=4, drop_last=False, shuffle=False)
test_ds_2 = Task2Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl_2 = DataLoader(test_ds_2, batch_size=32, num_workers=4, drop_last=False, shuffle=False)
test_ds_3 = Task3Dataset(test_data, root=TEST_PATH, return_filename=True)
test_dl_3 = DataLoader(test_ds_3, batch_size=32, num_workers=4, drop_last=False, shuffle=False)


if os.path.exists('/content/drive/MyDrive/model_0.9978/submission.csv'):
    f = open('/content/drive/MyDrive/model_0.9978/submission.csv', 'w', newline='')
    csv_writer = csv.writer(f)
    csv_writer.writerow(["filename", "label"])
else:
    f = open('/content/drive/MyDrive/model_0.9978/submission.csv', 'w', newline='')
    csv_writer = csv.writer(f)
    csv_writer.writerow(["filename", "label"])
    
model1 =  ResNet18_1().to(device)
model1.load_state_dict(torch.load("/content/drive/MyDrive/model_0.9978/model_task1"))
model2 =  ResNet18_2().to(device)
model2.load_state_dict(torch.load("/content/drive/MyDrive/model_0.9978/model_task2 "))
model3 =  ResNet18_3().to(device)
model3.load_state_dict(torch.load("/content/drive/MyDrive/model_0.9978/model_task3"))

model1.eval()
with torch.no_grad():
    for image, filenames in test_dl_1:
        image = image_change(image)
        image = image.to(device)

        pred = model1(image)
        pred = torch.argmax(pred, dim=1)

        for i in range(len(filenames)):
            print(str(pred[i].item()))
            csv_writer.writerow([filenames[i], str(pred[i].item())])
            time.sleep( 0.001 )
            
    del image, pred

model2.eval()
with torch.no_grad():
    for image, filenames in test_dl_2:
        image = image_change(image)
        image = image.to(device)

        pred = model2(image)
#         pred = torch.argmax(pred, dim=1)
        label = []
        for i in range(len(pred)) :
            character1 = torch.argmax(pred[i][0:36])
            character2 = torch.argmax(pred[i][36:72])
            ans = ""
            ans += alphabet[character1]
            ans += alphabet[character2]
            label.append(ans)
            
        for i in range(len(filenames)):
            csv_writer.writerow([filenames[i], label[i]])
#             time.sleep( 0.001 )
    del image, pred
    
model3.eval()
with torch.no_grad():
    for image, filenames in test_dl_3:
        image = image_change(image)
        image = image.to(device)

        pred = model3(image)
#         pred = torch.argmax(pred, dim=1)
        label = []
        for i in range(len(pred)) :
            character1 = torch.argmax(pred[i][0:36])
            character2 = torch.argmax(pred[i][36:72])
            character3 = torch.argmax(pred[i][72:108])
            character4 = torch.argmax(pred[i][108:144])
            ans = ""
            ans += alphabet[character1]
            ans += alphabet[character2]
            ans += alphabet[character3]
            ans += alphabet[character4]
            label.append(ans)

        for i in range(len(filenames)):
            csv_writer.writerow([filenames[i], label[i]])
            time.sleep( 0.001 )
            
    del image, pred

# for filename, _ in test_data:
#     if filename.startswith("task1") or filename.startswith("task3"):
#         csv_writer.writerow([filename, 0])
f.close()